**Трансформерная архитектура (GPT) "с нуля"**

In [15]:
import os
import keras_nlp
import keras

import tensorflow.data as tf_data
import tensorflow.strings as tf_stringss

**Подготовка**


In [16]:
from typing import List
from bs4 import BeautifulSoup
import requests
import os
def request_url(url: str) -> BeautifulSoup:
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup


def get_url_data(url: str) -> List[str]:
    soup = request_url(url)
    scrapped_text = []

    h1 = soup.h1.text.strip()
    p = soup.find_all('p')

    scrapped_text.append(h1)
    scrapped_text.extend([p_i.text.strip() for p_i in p])

    return scrapped_text

def get_data(url: str) -> str:
    soup = request_url(url)
    text = []

    text.extend([
        soup.h1.text.strip() + '.',
        soup.h2.text.strip() + '.',
        soup.article.p.text.strip()
    ])

    url_chapters = [link.get('href') for link in soup.find_all('a', class_='link')]

    for url in url_chapters:
        scrapped_text = get_url_data(url)
        text.extend(scrapped_text)

    text = ' '.join(text).lower()

    return text

In [17]:
def load_data(url: str, file_name: str, path_dir: str = 'data/') -> str:
    if os.path.isdir(path_dir) == False:
        os.mkdir(path_dir)
        print(f'Created {path_dir} directory')

    path_file = f'{path_dir}{file_name}'

    try:
        with open(path_file, 'r', encoding='utf-8') as file:
            text = file.read()

        print('Uploaded from', path_file)

    except:
        text = get_data(url)

        with open(path_file, 'w', encoding='utf-8') as file:
            file.write(text)

        print('Saved to', path_file)

    return text

In [18]:
text = load_data('https://hpmor.ru/', 'hpmor.txt')

Uploaded from data/hpmor.txt


In [19]:
path_train = 'data/hpmor_train.txt'
path_valid = 'data/hpmor_valid.txt'

In [20]:
from typing import Tuple, List
import random
def train_test_split(text: list, test_size: float) -> Tuple[List[str], List[str]]:
    random.shuffle(text)
    threshold = int((1 - test_size) * len(text))

    train = text[:threshold]
    test = text[threshold:]

    return train, test

def train_valid_test_split_save(text: str,
                                path_train: str,
                                path_valid: str,
                                path_test: str = None,
                                test_size: float = 0.25) -> None:

    if os.path.isfile(path_train) == False and os.path.isfile(path_valid) == False:

        text_split = [s.strip() for s in text.split('.')]

        train_text, valid_text = train_test_split(text_split, test_size)
        test_text = None

        if path_test != None:
            valid_text, test_text = train_test_split(valid_text, test_size)
            test_text = ' '.join(test_text)

        train_valid_test = [(path_train, train_text), (path_valid, valid_text), (path_test, test_text)]

        for (path, text) in train_valid_test:
            if path != None:
                with open(path, 'w', encoding='utf-8') as file:
                    file.write(' '.join(text))

        print('Splitted into:', len(train_text), 'train,', len(valid_text), 'valid and',
              len(test_text) if test_text != None else 0, 'test')

    else:
        print('Files already exist')

In [21]:
train_valid_test_split_save(text, path_train, path_valid)

Files already exist


In [22]:
from tensorflow.data import TextLineDataset
def preprocess(inputs: TextLineDataset):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels

In [23]:
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_STRING_LEN = 512
VOCAB_SIZE = 5000

BATCH_SIZE = 64
MIN_STRING_LEN = 512
SEQ_LEN = 128
EMBED_DIM = 256
FEED_FORWARD_DIM = 128
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000

EPOCHS = 100
NUM_TOKENS_TO_GENERATE = 80

In [28]:
raw_train_ds = (
    TextLineDataset(path_train)
    .filter(lambda x: tf.strings.length(x) > MIN_STRING_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

raw_val_ds = (
    TextLineDataset(path_valid)
    .filter(lambda x: tf.strings.length(x) > MIN_STRING_LEN)
    .batch(BATCH_SIZE)
)

In [25]:
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

In [26]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

In [29]:
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)


def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels


train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf_data.AUTOTUNE).prefetch(
    tf_data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf_data.AUTOTUNE).prefetch(
    tf_data.AUTOTUNE
)

In [41]:
inputs = keras.layers.Input(shape=(None,), dtype="int32")

embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)

for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)

outputs = keras.layers.Dense(VOCAB_SIZE)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer="adam", loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [42]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding_1       │ (None, None, 256)           │       1,312,768 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_decoder_2                │ (None, None, 256)           │         329,085 │
│ (TransformerDecoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_decoder_3                │ (None, None, 256)           │         329,085 │
│ (TransformerDecoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, None, 5000)          │       1,285,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,255,938 (12.42 MB)

 Trainable params: 3,255,938 (12.42 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 1.9055e-04 - loss: 8.5564 - val_accuracy: 0.0166 - val_loss: 8.3828
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0175 - loss: 8.2336 - val_accuracy: 0.0688 - val_loss: 8.2204
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0686 - loss: 7.9611 - val_accuracy: 0.0742 - val_loss: 8.0355
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0667 - loss: 7.6916 - val_accuracy: 0.0703 - val_loss: 7.8437
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0629 - loss: 7.4342 - val_accuracy: 0.0688 - val_loss: 7.6698
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0627 - loss: 7.1940 - val_accuracy: 0.0693 - val_loss: 7.5243
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.0631 - loss: 6.9649 - val_accuracy: 0.0742 - val_loss: 7.4059
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0694 - loss: 6.7411 - val_accuracy: 0.0781 - val_loss: 7.3128
Ep